## *Problem1.* Program a function that given a causal graph (in matrix form) computes the basis set of independences to be checked to assert if that graph is a feasible causal structure.

In [56]:
def Pearl_basis_set(graph):
    """The graph must be represented by a square matrix, where each line contains ones in the node parents.
    Returns the the minimum number of d-separation statements needed to completely specify the causal
    graph as descrived by Pearl (1988)"""
    basis_set = set() # given its name returning a set seems logical, a list could be used but could 
    for node1_index in range(len(graph)): # prone misunderstoods
        for node2_index in range(node1_index+1,len(graph)):
            if graph[node1_index][node2_index] or graph[node2_index][node1_index]: 
                continue # adjacent nodes
            else: # non-adjacent nodes
                parent_variables = []
                for i in range(len(graph)):
                    if graph[node1_index][i] or graph[node2_index][i]:
                        parent_variables.append(i)
                basis_set.add(tuple([node1_index,node2_index, tuple(parent_variables)]))
    return basis_set

### Give examples with different graphs and artificially generated data for both exercises.

In [50]:
import random

"""class very_simple_matrix():
    \"""Just redifines str assuming the object is a list of list\"""
    def __init__(self):
        self._list=list()
        
    def __getattr__(self, item):
        print(item)
        return object.__getattribute__(self._list,item) 
    
    
    def __str__(self):
        s = ""
        for i in self._list:
            for y in i:
                s += str(i)+" "
            s += "\n"
        return s"""

def gen_matrix(n,m=False):
    if not m:
        m = n
    mat = list() #very_simple_matrix()
    for rows in range(n):
        row=[]
        for column in range(m):
            if m == n: # one variable can not influence itself, to be completely accurate cycles must be avoided to
                row.append(0) # However, it's not needed to check the function works
            else:
                row.append(random.randint(0,1))
        print(row)
        mat.append(row)
    return mat

In [57]:
m = gen_matrix(5)

Pearl_basis_set(m)

[0, 0, 0, 0, 0]
[0, 0, 1, 1, 1]
[1, 1, 1, 0, 1]
[0, 0, 0, 1, 1]
[0, 0, 0, 1, 0]


{(0, 1, (2, 3, 4)), (0, 3, (3, 4)), (0, 4, (3,)), (2, 3, (0, 1, 2, 3, 4))}

## *Problem2.* Program a function that given a basis set and data for all the variables computes the independences given the data (using Pearson Correlation) and returns True if the independences are observed. The threshold of correlation to consider independent or dependent two variables is a parameter of the function.

In [ ]:
def Pearson_Correlation_computation(data, covid1, covid2):
    """Computes the correlation between the covariates 1 and 2, wich identified as colum id.
    There was no reasonable alibi to include -19"""
    a = 0 # sorry about the name, this is the upper part of the coefficient
    var1 = 0 
    var2 = 0
    mean1 = 0
    mean2 = 0
    for individual in data: #compute means
        mean1 += individual[covid1]/len(data)
        mean2 += individual[covid2]/len(data)
    for individual in data: # compute a var1 and var2
        a += (individual[covid1]-mean1)*(individual[covid2]-mean2)
        var1 += (individual[covid1]-mean1)**2
        var2 += (individual[covid2]-mean2)**2
    return(a/(var1**0.5 * var2**0.5))

In [ ]:
def independance_checker(basis, data, threshold):
    """Not taking into account parent variables"""
    for pair_of_variables in basis:
        if Pearson_Correlation_computation(data, pair_of_variables[0],pair_of_variables[1])> threshold:
            return 0
    return 1